#Install

SDK: https://github.com/anthropics/anthropic-sdk-python

In [ ]:
!pip install anthropic

import os
import json
import pandas as pd
import time
import re
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.4/846.4 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00



# This is formatted as code


### Import packages

In [ ]:
import pathlib
import textwrap


from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

  # Used to securely store your API key
from google.colab import userdata

### Setup your API key

Before you can use the Cluade API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://console.anthropic.com/settings/keys" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `CLUADE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `CLUADE_API_KEY` environment variable (the SDK will automatically pick it up from there).


In [ ]:
import os
from anthropic import Anthropic

client = Anthropic(
    # This is the default and can be omitted
    # Or use `os.getenv('finalproject-secret-key')` to fetch an environment variable.
    #CLUADE_API_KEY=userdata.get('finalproject-secret-key')
    api_key=userdata.get("ANTHROPIC_API_KEY"),
)


### Demo test for us

In [ ]:
snort_rule = '''alert tcp $HOME_NET [21,25,443,465,636,992,993,995,2484] -> $EXTERNAL_NET any ( msg:"SERVER-OTHER OpenSSL TLSv1.1 large heartbeat response - possible ssl heartbleed attempt"; flow:to_client,established; content:"|16 03 02|"; byte_jump:2,0,relative; content:"|18 03 02|",within 3,fast_pattern; byte_test:2,>,128,0,relative; metadata:policy max-detect-ips drop,policy security-ips drop,ruleset community; service:ssl; reference:cve,2014-0160; classtype:attempted-recon; sid:30781; rev:5; )'''

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": f"""
                          I'm going to give you a Snort rule. Read the Snort rule carefully, because I'm going to given you a task about it. Here is the Snort rule: <snort_rule>{snort_rule}</snort_rule>

                          First, find the techniques from MITRE ATT&CK that are most relevant to the Snort rule.

                          Then, answer the task, for each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.

                          Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.

                          Thus, the format of your overall response should look like what's shown between the <examples></examples> tags. Make sure to follow the formatting and spacing exactly.


                          <examples>
                          [
                            "sid": "2274",
                            "Technique ID": "T1110",
                            "Technique name": "Brute Force",
                            "Quotes": [
                              "\"PROTOCOL-POP login brute force attempt\"",
                              "track by_dst,count 30,seconds 30"
                            ],
                            "Explanation": "The rule is looking for excessive \"USER\" commands within a short period of time, which are common indicators of brute-force attacks targeting the POP3 service."
                          ]
                          </examples>

                          Do not include anything besides write the JSON.
                          """,
        }
    ],
    model="claude-2.1",
    temperature=0,
)


#print(message.content) # list

In [ ]:
text = str(message.content)
#text = str(message.content).split("\\n\\n")[1].split("', type")[0]
print(type(text))
print(text)

t_numbers = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)

print(t_numbers)

<class 'str'>
[ContentBlock(text='{\n    "Sid": "30781",\n    "Technique ID": "T1529",\n    "Technique Name": "System Services Discovery",\n    "Quotes": "\\"This rule detects attackers attempting to exploit the Heartbleed vulnerability in OpenSSL to read sensitive information from the server\'s memory.\\"",\n    "Explanation": "The rule is looking for signs of attackers trying to exploit the Heartbleed vulnerability, which allows reading sensitive data from memory. This aligns with the System Services Discovery technique in MITRE ATT&CK, which involves gathering information about services running on remote systems. The attempt to exploit Heartbleed to gather data is a form of system services discovery."\n}', type='text')]
['T1529']


### Data:
Our data will be taken from 162 snort rules that have already been manually labeled to techniques from MITRE ATT&CK.

In [ ]:
!git clone https://github.com/trickdeath0/Labeling_IDS_to_MITRE.git

Cloning into 'Labeling_IDS_to_MITRE'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 273 (delta 145), reused 232 (delta 107), pack-reused 0
Receiving objects: 100% (273/273), 819.99 KiB | 13.67 MiB/s, done.
Resolving deltas: 100% (145/145), done.


In [ ]:
data = pd.read_csv('/content/Labeling_IDS_to_MITRE/ground_truth.csv')
print(data.head())
rules_list = data['Rule']
true_labels = data['technique ids']

#print(data['Sid'][0+41])
print(f"\n{len(data)=}")

   Unnamed: 0  Sid                                               Rule  \
0           0  213  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
1           1  214  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
2           2  215  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
3           3  216  alert tcp $EXTERNAL_NET any -> $TELNET_SERVERS...   
4           4  233  alert tcp $EXTERNAL_NET any -> $HOME_NET 27665...   

  technique ids  
0     ['T1014']  
1     ['T1014']  
2     ['T1014']  
3     ['T1014']  
4     ['T1078']  


In [ ]:
def clean_response(text):
    text = text.data.replace(">", "").strip()  # Remove leading ">", whitespace
    try:
      text = text.replace("```json", "")
      text = text.replace("```", "")
    except:
      pass
    return text


# **Prompting without techniques guide (WTG):**
At this stage, the LLMs will receive a prompt that does not include the list of techniques from MITRE ATT&CK in order to examine the results of the models based on prior knowledge that has been trained. According to our request, the LLMs will classify the techniques according to the content of the rule.


**Prompt1**:

      prompt = f"""You are an information security expert. Your task is to label IDS rules for MITRE ATT&CK techniques based on your cybersecurity knowledge. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques from MITRE ATT&CK that are related to the rule.
      Try to search based on keywords and based on the knowledge you have. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
      Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
      Please don't write anything but the JSON. Rule: {snort_rule}"""


**prompt2**:

      prompt2 = f"""I'm going to give you a Snort rule. Read the Snort rule carefully, because I'm going to given you a task about it. Here is the Snort rule: <snort_rule>{snort_rule}</snort_rule>

      First, find the techniques from MITRE ATT&CK that are most relevant to the Snort rule.

      Then, answer the task, for each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.

      Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.

      Thus, the format of your overall response should look like what's shown between the <examples></examples> tags. Make sure to follow the formatting and spacing exactly.


      <examples>
        [
          "sid": "2274",
          "Technique ID": "T1110",
          "Technique name": "Brute Force",
          "Quotes": [
            "\"PROTOCOL-POP login brute force attempt\"",
            "track by_dst,count 30,seconds 30"
          ],
          "Explanation": "The rule is looking for excessive \"USER\" commands within a short period of time, which are common indicators of brute-force attacks targeting the POP3 service."
        ]
        </examples>

        Do not include anything besides write the JSON.
        """


**prompt3**:

        prompt3 = f"""You work in a company that deals with information security, your role in the company is to label techniques from MITRE ATT&CK to the rules of IDS systems. The labeling between a rule and a technique indicates that the attacker operated with a technique that you found to be suitable for the rule that alerted the IDS system. Now we will test your knowledge labeling IDS rules for MITRE ATT&CK techniques. For your task, you're going to have a single Snort IDS rule and you'll need to label the most relevant techniques from MITRE ATT&CK associated with the rule. From the rule you receive, your labeling should be based on your knowledge and the information found within the 'msg' in the rule received. For each technique you call the rule, include the following information as JSON format in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.  Note: The value of the 'Quotes' field should contain quotation marks from the data sets relevant to the mapped technique. The value of the 'Explanation' should be your explanation of why you decided to give the technique and how it relates to the rule. The 'Technique ID' should be the official MITRE technique ID.
        Please don't write anything but the JSON. Rule: {snort_rule}""")


**prompt4**

        prompt4 = f'''You are going to receive a Snort rule and your task is to find as many MITRE ATT&CK techniques as possible that are associated with the rule. Note: You should categorize the techniques to 1 or 2. Technique of type 1 is a technique that you can associate with the rule directly based on the rule. Technique of type 2 is a technique that can be associated with the rule indirectly, based on your knowledge and understanding. The categorization value should be the value 1 or 2, based on the explanation given above. The quotes field value should contain quotes from the rules data that are relevant to the technique mapped and they are the main reason you believe the mapping to this technique is correct. The explanation’s value should be your explanation for why you decided to give the technique and how it is associated with the rule. The technique id should be the official MITRE technique id. For each technique include the following information as JSON: sid, Technique id, Technique name, Categorization, Quotes, Explanation. After each rule I will provide you with, answer according to the provided format. Please do not write anything else but the JSON. Rule: {snort_rule}''')


In [ ]:
def wtg(snort_rule):

  prompt = f"""You are an information security expert. Your task is to label IDS rules for MITRE ATT&CK techniques based on your cybersecurity knowledge. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques from MITRE ATT&CK that are related to the rule.
              Try to search based on keywords and based on the knowledge you have. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
              Note: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
              Please don't write anything but the JSON. Rule: {snort_rule}"""


  message = client.messages.create(
      max_tokens=1024,
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="claude-2.1",
      temperature=0,
  )

  return message

In [ ]:
rule_dict = {}
max_retries = 3  # Maximum number of retries

for index, rule in enumerate(rules_list):
    retries = 0
    while retries < max_retries:
        try:
            res = wtg(rule)
            text = str(res.content)
            # Check if the text contains the desired pattern
            t_numbers = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)
            if t_numbers:  # If the pattern is found
                rule_dict[data['Sid'][index]] = to_markdown(text)
                break  # Break out of the retry loop if successful
            else:
                print("Desired pattern not found in the text. Retrying...")
                retries += 1
                time.sleep(1)  # Wait for a short duration before retrying

            time.sleep(15) # remove after we have money :)
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying... ({retries}/{max_retries})")
                time.sleep(1)  # Wait for a short duration before retrying
            else:
                print("Max retries reached. Unable to process this rule.")

# If sending fails, attempt to send again
try:
    # Code to send data
    pass
except Exception as e:
    print(f"Sending failed: {e}")
    # Retry sending here


An error occurred: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of requests has exceeded your rate limit. Please try again later or contact sales@anthropic.com to discuss your options for a rate limit increase.'}}
Retrying... (1/3)
An error occurred: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of requests has exceeded your rate limit. Please try again later or contact sales@anthropic.com to discuss your options for a rate limit increase.'}}
Retrying... (1/3)
An error occurred: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'Number of requests has exceeded your rate limit. Please try again later or contact sales@anthropic.com to discuss your options for a rate limit increase.'}}
Retrying... (2/3)
An error occurred: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying... (1/3)
An error occurred: Erro

Write to csv

In [ ]:

# Define the field names
field_names = ["Sid", "Response", "Technique_id", "True_labels"]

# Open the CSV file in write mode (truncating any existing content)
with open("prompting_without_techniques_guide.csv", "w", newline="") as csvfile:
    # Create a DictWriter object with the specified field names
    writer = csv.DictWriter(csvfile, fieldnames=field_names)

    # Write the header row
    writer.writeheader()

    # Extract relevant data from each item and write it as a dictionary
    counter = 0
    for key, value in rule_dict.items():
      text = clean_response(value)
      technique_ids = []

      if "'Sid" in text:
        # Define a regex pattern to switch single quotes to double quotes
        pattern = re.compile(r"((^|\s)'((?:[^'\\]|\\.)*)'(?=[\s.,:;!?)]))|(:\s*'((?:[^'\\]|\\.)+)')")
        # Switch single quotes to double quotes
        text = pattern.sub(lambda x: x.group().replace("'", '"'), text)

        pattern = re.compile(r'"\S+"[\s\.]|\s"[\w\s]*"\s')
        text = re.sub(pattern, "", text)

      # Extracting "TXXXX" numbers using regular expression
      technique_ids = re.findall(r'[\'\"](T\d+(?:\.\d+)?)', text)

      # Extracting "Sid"
      match = re.search(r'[\'\"][s|S]id[\'\"]: [\'\"](\d+)[\'\"]', text)
      if match:
          sid_number = match.group(1)


      # Assuming each item has all necessary fields:
      insertRow = {
          "Sid": sid_number,
          "Response": text,
          "Technique_id": technique_ids,  # Handle potential absence
          "True_labels": true_labels[counter],
      }
      writer.writerow(insertRow)
      counter += 1


# Evaluation


*   Persicion
*   Recall
*   F-1



In [ ]:
import ast
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

def evaluation(true_labels, predicted_labels):

  recall = []
  precision = []
  f1 = []


  for i in range(len(true_labels)):
    trueList = ast.literal_eval(true_labels[i])
    predList = ast.literal_eval(predicted_labels[i])
    # Extract only the 'TXXXX' part from each string in the list
    predList = [item.split('.')[0] if '.' in item else item for item in predList]
    intersection = set(trueList).intersection(set(predList))
    #print(list(intersection))

    if (len(predList) != 0):
      recall.append(len(intersection) / len(trueList))
      precision.append(len(intersection) / len(predList))
      try:
        f1.append((2 * precision[i] * recall[i]) / (recall[i] + precision[i]))
      except:
        f1.append(0)

  # Avg.
  average_recall = sum(recall) / len(recall)
  average_precision = sum(precision) / len(precision)
  average_f1 = sum(f1) / len(f1)

  print("Metric    |   Score")
  print("-------------------")
  print(f"Precision |   {average_precision:.2f}")
  print(f"Recall    |   {average_recall:.2f}")
  print(f"F1 Score  |   {average_f1:.2f}")

In [ ]:
loadData = pd.read_csv('prompting_without_techniques_guide.csv')
true_labels = loadData['True_labels']
predicted_labels = loadData['Technique_id']

evaluation(true_labels, predicted_labels)


Metric    |   Score
-------------------
Precision |   0.38
Recall    |   0.32
F1 Score  |   0.33


# **Prompting with techniques guide (TG):**
In the next step, we will provide the LLMs with the list of all the techniques from MITRE ATT&CK, to guarantee that the models are targeted to the present techniques, even the infrequently used ones. Each technique will include the technique number, the name of the technique and its description. The techniques will be provided to the models in the form of batches (due to the memory limit of the models) and after each batch we will ask him to classify the appropriate techniques from the list he received (if exist), finally we will unite the model's answers for each individual rule.


pre colletion data for TG

In [ ]:
def recursive_enter(path: str, file_list: list = None) -> list:
    if file_list is None:
        file_list = []

    try:
        os.chdir(path)  # Change path

        items = os.listdir()  # List everything in the directory
        for item in items:
            full_path = os.path.join(path, item)

            if full_path.endswith(".json"):
                with open(full_path) as f:
                    file_list.append(json.load(f))

    except Exception as e:
        print(f"An error occurred: {e}")

    return file_list

tacticFolder = "/content/Labeling_IDS_to_MITRE/Extract data from MITRE ATTACK/techniques_split"
file_list = []
MITRE_Technique = recursive_enter(tacticFolder, file_list)
print(len(MITRE_Technique))
os.chdir("/content/")

11


In [ ]:
def tg(snort_rule, techniques):

  prePrompt = f"""You are an information security expert. Now I will provide you information about techniques from MITRE ATT&CK, you will use the information for a task you will receive later. Do not reply to the information you receive."""

  dataPrompt = f"The information:\n {str(techniques)}"

  response_data = f"""Your task is to label IDS rules for MITRE ATT&CK techniques based on the information I have provided you. For the task, you are going to get a single Snort IDS rule and you will need to return the most relevant techniques associated with the rule from the information I provided you only.
   Note 1: There is not necessarily a suitable technique in the information, return a technique if and only if it has an unambiguous relationship to the provided rule, if not return an empty JSON. For each technique include the following information as JSON in this order: 'Sid', 'Technique ID', 'Technique Name', 'Quotes', 'Explanation'.
   Note 2: The value of the citation field should contain quotation marks from the data sets relevant to the mapped technique are the main reason you chose this technique to be correct. The value of the explanation should be your explanation of why you decided to give the technique and how it relates to the rule. The technique ID should be the official MITRE technique ID.
   Please don't write anything but the JSON. Rule: {snort_rule}"""

  tg_data_list = prePrompt + dataPrompt + response_data

  message = client.messages.create(
      max_tokens=1024,
      messages=[
          {
              "role": "user",
              "content": tg_data_list,
          }
      ],
      model="claude-2.1",
      temperature=0,
  )

  return message




In [ ]:
import csv
import re

headersCSV_TG = ["Sid", "Response", "Technique_id", "True_labels"]

# Initial write to csv with header
with open('prompting_with_techniques_guide.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headersCSV_TG)
    writer.writeheader()

def appendToCSV(rows_data, counter) -> None:
    '''
    rows_data -> {213: [<IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, ...]}
    '''
    # Open the CSV file in append mode to add new rows
    with open('prompting_with_techniques_guide.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headersCSV_TG)

        # Loop through each row and write data
        for row, value in rows_data.items():
            technique_ids = []
            response_text = ""
            for i in value:
                text = clean_response(i)
                response_text += text
                #print(text)

                try:
                    # Extracting "TXXXX" numbers using regular expression
                    technique_ids.extend(re.findall(r': [\'\"](T\d+(?:\.\d+)?)', text))
                except Exception as e:
                    print(f"Error extracting technique IDs: {e}")

            insertRow = {
                "Sid": row,
                "Response": response_text,
                "Technique_id": technique_ids,
                "True_labels": true_labels[counter]
            }

            # Write the row to the CSV file
            writer.writerow(insertRow)


In [ ]:
def tg_split_data(rules_list_index, index):

  for rule in rules_list_index:
    print(f"index {index} \t Sid: {data['Sid'][index]}")

    tg_dict = {}
    count = 0 #####
    for batch in MITRE_Technique: # 11 files
      res = tg(rule, batch)
      sid = data['Sid'][index]
      if sid not in tg_dict:
        tg_dict[sid] = []
        tg_dict[sid].append(to_markdown(str(res.content)))
      else:
        tg_dict[sid].append(to_markdown(str(res.content)))
      print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{count}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") #####
      print(to_markdown(str(res.content)))
      count += 1 #######
      time.sleep(15)

    # Write to csv
    appendToCSV(tg_dict, index)
    index += 1



In [ ]:
rules_list1 = rules_list[:5] # index 0-1
#rules_list2 = rules_list[2:4] # index 2-3

tg_split_data(rules_list1, 0)
# tg_split_data(rules_list2, 2)

index 0 	 Sid: 286
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~1~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~3~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~4~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~5~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~6~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~7~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.core.display.Markdown object>
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~8~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<IPython.cor

# Evaluation


*   Persicion
*   Recall
*   F-1



In [ ]:
loadData = pd.read_csv('prompting_with_techniques_guide.csv')
true_labels = loadData['True_labels']
predicted_labels = loadData['Technique_id']

evaluation(true_labels, predicted_labels)


# Original

## Same code without split data (long running)

In [ ]:
for index, rule in enumerate(rules_list[:5]): # 162 rules
  print(f"index {index} \t Sid: {data['Sid'][index]}")

  # Splice from index 1 to index 3 (exclusive)
  start_index = 0
  #end_index = 1
  spliced_dict = {k: MITRE_Technique[k] for k in list(MITRE_Technique.keys())[start_index:]}


  tg_dict = {}
  count = 0 #####
  for tactics, techniques_inside in spliced_dict.items(): # 14 folders  #################### return to MITRE_Technique.items()
    print(f"Tactic check: {tactics}")
    for techniques in techniques_inside[:1]:
      print(f"inside folder tactic: {techniques}\n")
      res = tg(rule, tactics, techniques)
      sid = data['Sid'][index]
      if sid not in tg_dict:
        tg_dict[sid] = []
        tg_dict[sid].append(to_markdown(str(res.content)))
      else:
        tg_dict[sid].append(to_markdown(str(res.content)))
      print(f"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{count}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~") #####
      print(to_markdown(str(res.content)))
      count += 1 #######
      time.sleep(20)

  # Write to csv
  appendToCSV(tg_dict, index)


index 0 	 Sid: 286
Tactic check: Persistence
inside folder tactic: {'T1547.014': ['Active Setup', 'Adversaries may achieve persistence by adding a Registry key to the Active Setup of the local machine'], 'T1547.015': ['Login Items', 'Adversaries may add login items to execute upon user login to gain persistence or escalate privileges'], 'T1037': ['Boot or Logon Initialization Scripts', 'Adversaries may use scripts automatically executed at boot or logon initialization to establish persistence'], 'T1037.001': ['Logon Script (Windows)', 'Adversaries may use Windows logon scripts automatically executed at logon initialization to establish persistence'], 'T1037.002': ['Login Hook', 'Adversaries may use a Login Hook to establish persistence executed upon user logon'], 'T1037.003': ['Network Logon Script', 'Adversaries may use network logon scripts automatically executed at logon initialization to establish persistence'], 'T1037.004': ['RC Scripts', 'Adversaries may establish persistence by 

KeyboardInterrupt: 

In [ ]:
#print(to_markdown(res.text).data)

print(tg_dict)
# print(tg_dict[213][0].data)
for key, value in tg_dict.items():
  for i in value:
    text = clean_response(i)
    print(text)

{382: [<IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>, <IPython.core.display.Markdown object>]}
[ContentBlock(text='[]', type='text')]
[ContentBlock(text='{}', type='text')]
[ContentBlock(text='[]', type='text')]
[ContentBlock(text='{}', type='text')]
[ContentBlock(text='{}', type='text')]
[ContentBlock(text='[]', type='text')]
[ContentBlock(text='[]', type='text')]
[ContentBlock(text='{}', type='text')]
[ContentBlock(text='{}', type='text')]
[ContentBlock(text='{}', type='text')]
[ContentBlock(text='[]', type='text')]
